# Market depth (order book)


In [ ]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=10)
# util.logToConsole()

In [ ]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)


def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


ticker.updateEvent += onTickerUpdate

IB.sleep(15)

In [ ]:
ib.cancelMktDepth(contract)

In [ ]:
ib.positions(account="U10394496")

In [ ]:
def print_all_openorders(sym="NQ"):
    print("All Open Orders")
    print(f"symbol\tpermId\t\tstatus\t\taction\tfilled\tremaining\tlmtPrice")

    trades = ib.reqAllOpenOrders()

    trades.sort(key=lambda trade: trade.order.lmtPrice)

    for trade in trades:
        orderstatus = trade.orderStatus
        order = trade.order

        if trade.contract.symbol != sym:
            continue

        print(
            f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
        )


def print_trades(trades):
    print(f"symbol\tpermId\t\tstatus\t\taction\tfilled\tremaining\tlmtPrice")

    trades.sort(key=lambda trade: trade.order.lmtPrice)

    for trade in trades:
        orderstatus = trade.orderStatus
        order = trade.order

        print(
            f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
        )


def get_all_openorders(sym="NQ"):
    trades = ib.reqAllOpenOrders()
    trades.sort(key=lambda trade: trade.order.lmtPrice)

    trades = [trade for trade in trades if trade.contract.symbol == sym]
    return trades


# print_all_openorders()


def print_order(o):
    if o is None:
        print(o)
        return

    order = o.order
    contract = o.contract
    orderStatus = o.orderStatus

    print(f"symbol\tpermId\t\tstatus\t\taction\tfilled\tremaining\tlmtPrice")

    print(
        f"{contract.symbol}\t{order.permId}\t{orderStatus.status}\t{order.action}\t{orderStatus.filled}\t{orderStatus.remaining}\t\t{order.lmtPrice}\t"
    )


# print_order(takeprofit)

In [ ]:
openorders = get_all_openorders()
print([o for o in openorders if o.order.permId == 740241587])

In [ ]:
# initialize seed parameters at beginning only once
opentobuy = None


takeprofit = [o for o in openorders if o.order.permId == 740241587]


In [196]:
# L1 = buy @ mid - 10 tickets , tp + 10 ticks

# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)


qty_target = -7

# get current position of contract nq24
position = ib.positions(account="U10394496")
future = position[0]

openorders = get_all_openorders(sym="NQ")

print("Open to Buy Order:")
print(opentobuy.log[-1])
print()
print("Take Profit Order:")
print(takeprofit)
print()
print(f"Ask Price {ticker.domAsks[0].price}")
print(f"Bid Price {ticker.domBids[0].price}")
print()

if future.position == qty_target:
    print("Position is at target quantity / {}, {}".format(future.position, qty_target))
    # check for initiation order (L1.buy)
    if opentobuy is None:
        print("Initiation Order does not exist")
        print()

        print("Placing L1.buy")
        action = "BUY"
        totalQuantity = 1
        opentobuy_lmtPrice = ticker.domBids[0].price
        lmtPrice = opentobuy_lmtPrice
        print("Setting open to buy order:")
        print(f"{action}, totalQuantity {totalQuantity}, lmtPrice {lmtPrice}")
        print("Setting takeprofit -> None")
        print()

        limitbuy = LimitOrder(
            action=action,
            totalQuantity=totalQuantity,
            lmtPrice=lmtPrice,
            account="U10394496",
        )
        # opentobuy = ib.placeOrder(contract, limitbuy)
        # ib.sleep(1)
    else:
        print("Initiation Order exists...")

    print(opentobuy)
    print()

if future.position > qty_target:
    print("Future.position {} > {} qty_target".format(future.position, qty_target))
    print()
    # check for take profit order (L1.sell)
    if takeprofit is None:
        print("TP Order does not exist")
        print("Placing TP Order")
        print()

        # place L1.sell
        action = "SELL"
        totalQuantity = 1
        lmtPrice = opentobuy.orderStatus.avgFillPrice + 10 * 0.25
        print(f"{action}, totalQuantity {totalQuantity}, lmtPrice {lmtPrice}")
        limitsell = LimitOrder(
            action=action,
            totalQuantity=totalQuantity,
            lmtPrice=lmtPrice,
            account="U10394496",
        )

        # takeprofit = ib.placeOrder(contract, limitsell)
        # ib.sleep(1)
    else:
        print("Take Profit Order exists...")
        print()

    print(takeprofit)
    print()

nq_position = [
    pos for pos in ib.positions(account="U10394496") if pos.contract.symbol == "NQ"
][0]
print(
    f"Symbol {nq_position.contract.symbol} Position {nq_position.position} avgCost {nq_position.avgCost/20}"
)
print()
futures_port = [p for p in ib.portfolio() if p.contract.symbol == "NQ"][0]
print(
    f"RealizedPNL {futures_port.realizedPNL} UnrealizedPNL {futures_port.unrealizedPNL} MarketValue {futures_port.marketValue/1e6}mm"
)

Open to Buy Order:
TradeLogEntry(time=datetime.datetime(2024, 4, 29, 8, 30, 40, 406788, tzinfo=datetime.timezone.utc), status='Filled', message='', errorCode=0)

Take Profit Order:
LimitOrder(action='SELL', totalQuantity=1, lmtPrice=17905.75, account='U10394496')

Ask Price 17895.25
Bid Price 17894.5

Future.position -6.0 > -7 qty_target

Take Profit Order exists...

LimitOrder(action='SELL', totalQuantity=1, lmtPrice=17905.75, account='U10394496')

Symbol NQ Position -6.0 avgCost 17892.076428275

RealizedPNL -3175.17 UnrealizedPNL -110.83 MarketValue -2.14716mm


In [195]:
print(takeprofit)

LimitOrder(action='SELL', totalQuantity=1, lmtPrice=17905.75, account='U10394496')


In [ ]:
# opentobuy = ib.placeOrder(contract, opentobuy)
ib.sleep(1)
opentobuy.log

In [ ]:
print(opentobuy.orderStatus.status)

In [186]:
def print_all_openorders(sym="NQ"):
    print("All Open Orders:\n")
    print(f"symbol\tpermId\t\tstatus\t\taction\tfilled\tremaining\tlmtPrice")

    trades = ib.reqAllOpenOrders()

    # order every Trade object by its order.lmtPrice field
    trades.sort(key=lambda trade: trade.order.lmtPrice)

    for trade in trades:
        orderstatus = trade.orderStatus
        order = trade.order

        if trade.contract.symbol != sym:
            continue

        print(
            f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
        )


def get_all_openorders(sym="NQ"):
    trades = ib.reqAllOpenOrders()
    trades.sort(key=lambda trade: trade.order.lmtPrice)

    # or return trades where contract.conId == 620730920
    trades = [trade for trade in trades if trade.contract.symbol == sym]
    return trades


trades = get_all_openorders()

print_all_openorders()

All Open Orders:

symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1528433802	Submitted	BUY	0.0	3.0		17526.0	
NQ	1436152242	Submitted	BUY	0.0	1.0		17547.25	
NQ	1786380178	Submitted	BUY	0.0	1.0		17623.25	
NQ	1786380182	Submitted	BUY	0.0	1.0		17650.0	
NQ	1056387955	PreSubmitted	SELL	0.0	2.0		20880.0	


In [ ]:
# canceled orders
print(f"permId\t\taction\t\ttotalQuantity\tfilledQuantity\t\tlmtPrice")

closedOrders = ib.orders()
[
    print(
        f"{order.permId}\t{order.action}\t\t{order.totalQuantity}\t\t{order.filledQuantity}\t\t\t{order.lmtPrice}"
    )
    for order in closedOrders
    if order.filledQuantity == 0
]

In [ ]:
ib.disconnect()